<a href="https://colab.research.google.com/github/JamionW/Advanced-Analysis-of-Algorithms/blob/master/Master_Code_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## This is the master notebook.

In [ ]:
!pip install osmnx # install the osmnx module


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
# IMPORTS

import pandas as pd
import geopandas as gpd
import fiona
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import nearest_points, linemerge, transform
from shapely.geometry import Point, LineString
from geopandas.tools import sjoin_nearest
from scipy.spatial import cKDTree
from collections import defaultdict
from pyproj import CRS, Transformer
from functools import partial

In [ ]:
def find_nearest_linestring_efficient(gdf_points, gdf_lines, max_distance):
    """
    Find the nearest linestring for each point, up to a maximum distance.
    Uses spatial indexing for efficiency.

    :param gdf_points: GeoDataFrame with point geometries (in UTM)
    :param gdf_lines: GeoDataFrame with linestring geometries (in UTM)
    :param max_distance: Maximum distance to consider (in meters)
    :return: GeoDataFrame with points matched to nearest linestrings
    """
    # Use sjoin_nearest to find the nearest linestring for each point
    joined = sjoin_nearest(gdf_points, gdf_lines, max_distance=max_distance, how='left')

    # Calculate the actual distances
    joined['distance'] = joined.apply(lambda row: row['geometry'].distance(gdf_lines.loc[row['index_right'], 'geometry'])
                                      if pd.notnull(row['index_right']) else None, axis=1)

    # Remove matches beyond max_distance (should be unnecessary due to max_distance in sjoin_nearest, but just in case)
    joined = joined[joined['distance'] <= max_distance]

    # Drop unnecessary columns
    result = joined.drop(columns=['index_right', 'distance'])

    print(f"Matched {result.notna().any(axis=1).sum()} out of {len(result)} points")

    return result


# Dataset imports

In [ ]:
# ADDRESSES

# Read in addresses
# this takes about 20 minutes for the State of Tennessee
# less than a minute for Chattanooga

# Read the GeoJSON file into a GeoDataFrame
#address_df = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/tennessee.geojson')

# Chattanooga, for testing
address_df = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/chattanooga.geojson')


In [ ]:
# SVI

# Path to .gdb file
gdb_file = "/content/drive/MyDrive/Colab Notebooks/data/SVI2022_TENNESSEE_tract.gdb"

# List all the layers in the .gdb file
layers = fiona.listlayers(gdb_file)
print("Layers in the geodatabase:", layers)

# Read the desired layer
svi_df = gpd.read_file(gdb_file, layer='SVI2022_TENNESSEE_tract')


Layers in the geodatabase: ['SVI2022_TENNESSEE_tract']


In [ ]:
# ROADS

# Import shapefiles
# https://www.census.gov/cgi-bin/geo/shapefiles/index.php

# documentation here: https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2023/TGRSHP2023_TechDoc.pdf

# Open the shapefile as a Fiona collection
with fiona.open('/content/drive/MyDrive/Colab Notebooks/data/tl_2023_47065_roads.shp') as collection:
    # Create a GeoDataFrame from the collection
    roads_df = gpd.GeoDataFrame.from_features(collection)


In [ ]:
# AMENITIES

city = "Chattanooga, Tennessee, USA"
tags = {'amenity': ['school', 'hospital', 'library'],
        'shop': 'supermarket'}

amenities = ox.features_from_place(city, tags=tags)

### Define Coordinate Reference Systems

In [ ]:
# Check the original CRS
print("Original CRS:", roads_df.crs)

# If the CRS is None, set it to WGS84 (assuming that's what it should be)
if roads_df.crs is None:
    roads_df.set_crs(epsg=4326, inplace=True)

# Define the target CRS (UTM zone 18N)
target_crs = CRS("EPSG:32618")

# Perform the transformation
roads_df_transformed = roads_df.to_crs(target_crs)

# Check the new CRS
print("New CRS:", roads_df_transformed.crs)

# Print a sample of the transformed geometries
print("Sample of transformed geometries:")
print(roads_df_transformed['geometry'].head())

Original CRS: None
New CRS: EPSG:32618
Sample of transformed geometries:
0    LINESTRING (-438047.238 3925110.410, -438015.6...
1    LINESTRING (-438047.238 3925110.410, -437997.8...
2    LINESTRING (-438379.567 3925013.438, -438362.5...
3    LINESTRING (-434083.556 3936175.511, -434066.9...
4    LINESTRING (-436868.331 3941879.564, -436868.2...
Name: geometry, dtype: geometry


In [ ]:
print("Bounding box of the data:")
print(roads_df.total_bounds)

Bounding box of the data:
[-85.469528  34.982924 -84.94233   35.459232]


In [ ]:
# Define the coordinate reference systems
latlong_crs = CRS("EPSG:4326")  # WGS84 lat/long
utm_crs = CRS("EPSG:32618")  # UTM zone 18N

address_df = address_df.to_crs(utm_crs)
svi_df = svi_df.to_crs(utm_crs)
amenities = amenities.to_crs(utm_crs)

print(address_df.crs)
print(svi_df.crs)
print(roads_df_transformed.crs)
print(amenities.crs)

EPSG:32618
EPSG:32618
EPSG:32618
EPSG:32618


### Data Engineering: SVI Filtering

In [ ]:
# Remove all columns from the svi_df dataframe except "geometry","STATE","ST_ABBR","COUNTY","FIPS","LOCATION","AREA_SQMI", and "RPL_THEME4".

svi_df = svi_df[["geometry","STATE","ST_ABBR","COUNTY","FIPS","LOCATION","AREA_SQMI", "RPL_THEME4"]]


### Data Engineering: Amenities cleanup

In [ ]:
# Remove rows where the 'amenity' column is null
joined_amenities_df = amenities.dropna(subset=['amenity'])
amenities = amenities.dropna(subset=['amenity'])
print(f"Number of amenities after removing null 'amenity' values: {len(amenities)}")

Number of amenities after removing null 'amenity' values: 101


In [ ]:
# Distribution of valid values in the amenity column of joined_amenities_df

print(amenities['amenity'].value_counts())


amenity
school        85
hospital      10
library        5
restaurant     1
Name: count, dtype: int64


In [ ]:
# Filter amenities
amenity_types = ['school', 'library', 'hospital']
filtered_amenities_df = amenities[amenities['amenity'].isin(amenity_types)]

print(f"Number of filtered amenities: {len(filtered_amenities_df)}")


Number of filtered amenities: 100


### Feature Engineering: Address Density

In [ ]:
### This is to engineer the feature for address density

# Extract coordinates from the geometry column
coords = np.array(list(address_df.geometry.apply(lambda x: (x.x, x.y))))

# Build the KD-tree
tree = cKDTree(coords)

# Set the buffer distance (e.g., 1000 meters)
buffer_distance = 1000

# Query the tree for all points within the buffer distance
indices = tree.query_ball_point(coords, r=buffer_distance)

# Count the number of neighbors, excluding the point itself
address_df['address_density'] = [len(idx) - 1 for idx in indices]

# Optionally, normalize the density
max_density = address_df['address_density'].max()
address_df['normalized_density'] = address_df['address_density'] / max_density

# Print some statistics
print(address_df['address_density'].describe())

count    102761.000000
mean       1235.861251
std         643.406327
min           0.000000
25%         729.000000
50%        1196.000000
75%        1663.000000
max        3676.000000
Name: address_density, dtype: float64


In [ ]:
#Join on geometry attributes

joined_svi_address_df = gpd.overlay(address_df, svi_df, how='intersection')

## Place Addresses and Amenities on a graph

In [ ]:
# Function to create a graph from a GeoDataFrame of roads
def create_graph_from_roads(roads_gdf):
    G = nx.Graph()
    for idx, row in roads_gdf.iterrows():
        if row.geometry.geom_type == 'LineString':
            start = row.geometry.coords[0]
            end = row.geometry.coords[-1]
            G.add_edge(start, end, geometry=row.geometry, length=row.geometry.length)
        elif row.geometry.geom_type == 'MultiLineString':
            merged = linemerge(row.geometry)
            if merged.geom_type == 'LineString':
                start = merged.coords[0]
                end = merged.coords[-1]
                G.add_edge(start, end, geometry=merged, length=merged.length)
            else:
                for line in merged.geoms:
                    start = line.coords[0]
                    end = line.coords[-1]
                    G.add_edge(start, end, geometry=line, length=line.length)
    return G

# Create the graph
G = create_graph_from_roads(roads_df_transformed)

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

def add_points_to_graph_optimized(G, gdf_points, n_jobs=None):
    # Determine the number of jobs
    if n_jobs is None or n_jobs <= 0:
        n_jobs = multiprocessing.cpu_count()  # Use all available CPUs

    # Extract node coordinates and create a KD-tree
    nodes = np.array([(n[0], n[1]) for n in G.nodes if isinstance(n, tuple) and len(n) == 2])
    tree = cKDTree(nodes)

    # Extract point coordinates
    points = np.array([(p.x, p.y) for p in gdf_points.geometry])

    def process_point(idx, point):
        try:
            # Find nearest node
            distance, nearest_index = tree.query(point, k=1)
            nearest_node = tuple(nodes[nearest_index])

            # Create point node
            point_node = f"point_{idx}"
            point_data = gdf_points.iloc[idx].to_dict()

            return point_node, nearest_node, distance, point, point_data
        except Exception as e:
            print(f"Error processing point {idx}: {str(e)}")
            return None

    # Use parallel processing to handle points
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        futures = [executor.submit(process_point, idx, point) for idx, point in enumerate(points)]

        for future in as_completed(futures):
            result = future.result()
            if result:
                point_node, nearest_node, distance, point, point_data = result
                G.add_node(point_node, geometry=point, point_data=point_data)
                G.add_edge(point_node, nearest_node, length=distance)

    return G

# Usage
G = add_points_to_graph_optimized(G, joined_svi_address_df)

In [ ]:
# Need to do this in order to work with the Point geographies within the amenities polygons

filtered_amenities_df = filtered_amenities_df.copy()
filtered_amenities_df.loc[:, 'geometry'] = filtered_amenities_df.geometry.centroid

In [ ]:
# Add the amenities to the Graph

G = add_points_to_graph_optimized(G, filtered_amenities_df)

### Further Feature Engineering

In [ ]:
import networkx as nx
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def engineer_features(df, G):
    # Start with relevant numerical features
    features = df[['address_density', 'normalized_density', 'AREA_SQMI']].copy()

    # One-hot encode categorical variables
    categorical_features = ['STATE', 'ST_ABBR', 'COUNTY']
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_features = encoder.fit_transform(df[categorical_features])

    # Check which method is available and use it
    if hasattr(encoder, 'get_feature_names_out'):
        feature_names = encoder.get_feature_names_out(categorical_features)
    else:
        feature_names = encoder.get_feature_names(categorical_features)

    encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

    # Combine numerical and encoded categorical features
    features = pd.concat([features, encoded_df], axis=1)

    # Extract amenity distances from the graph
    amenity_types = ['grocery', 'hospital', 'school', 'park']  # Add or remove types as needed
    max_distance = 1000000  # A large number to represent "unreachable"

    for amenity_type in amenity_types:
        features[f'distance_to_nearest_{amenity_type}'] = df.apply(
            lambda row: get_nearest_amenity_distance(G, (row.geometry.x, row.geometry.y), amenity_type, max_distance),
            axis=1
        )

    # Calculate local network characteristics
    for node in G.nodes():
        if 'geometry' in G.nodes[node]:
            features.at[node, 'local_edge_density'] = len(list(G.edges(node))) / G.graph['area']
            features.at[node, 'avg_neighbor_degree'] = nx.average_neighbor_degree(G, nodes=[node])[node]

    return features

def get_nearest_amenity_distance(G, source, amenity_type, max_distance):
    try:
        distances = [nx.shortest_path_length(G, source, target, weight='length')
                     for target in G.nodes()
                     if G.nodes[target].get('type') == amenity_type]
        return min(distances) if distances else max_distance
    except nx.NetworkXNoPath:
        return max_distance

# Apply the feature engineering
feature_df = engineer_features(joined_svi_address_df, G)

print(feature_df.head())

# Store RPL_THEME4 separately as our reference
reference_svi = joined_svi_address_df['RPL_THEME4']

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Implement Edge Accessibility (in development, not working, need parallelism)

In [ ]:
# import numpy as np
# from concurrent.futures import ProcessPoolExecutor, as_completed

# def calculate_ea(G, origins, destinations, omega=1.28, max_workers=None):
#     """
#     Calculate Edge Accessibility for the graph G.

#     :param G: NetworkX graph
#     :param origins: List of origin nodes
#     :param destinations: List of destination nodes
#     :param omega: Impedance parameter (default: 1.28)
#     :param max_workers: Number of parallel processes to use
#     :return: Dictionary of EA values for each edge
#     """
#     def process_edge(edge):
#         u, v = edge
#         original_length = G[u][v]['length']

#         # Remove the edge
#         G.remove_edge(u, v)

#         ea_value = 0
#         for origin in origins:
#             for dest in destinations:
#                 # Skip if origin and destination are the same
#                 if origin == dest:
#                     continue

#                 try:
#                     # Calculate shortest path length with the edge removed
#                     length_without_edge = nx.shortest_path_length(G, origin, dest, weight='length')

#                     # Get origin and destination attributes
#                     origin_data = G.nodes[origin].get('point_data', {})
#                     dest_data = G.nodes[dest].get('point_data', {})

#                     # Extract multipliers and weights
#                     mu_i = origin_data.get('address_density', 1)  # Use address density as multiplier
#                     rho_i = origin_data.get('normalized_density', 1)  # Use normalized density as weight
#                     gamma_j = 1  # Assuming no destination multiplier
#                     phi_j = dest_data.get('amenity_importance', 1)  # You'll need to add this attribute to amenities

#                     # Calculate EA contribution
#                     ea_value += (mu_i * gamma_j * rho_i * phi_j) * (1 / (length_without_edge ** omega))
#                 except nx.NetworkXNoPath:
#                     # If no path exists, contribution is 0
#                     pass

#         # Restore the edge
#         G.add_edge(u, v, length=original_length)

#         return edge, ea_value

#     edges = list(G.edges())
#     ea_values = {}

#     with ProcessPoolExecutor(max_workers=max_workers) as executor:
#         future_to_edge = {executor.submit(process_edge, edge): edge for edge in edges}
#         for future in as_completed(future_to_edge):
#             edge, value = future.result()
#             ea_values[edge] = value

#     return ea_values

# Testing only: filter and export

In [ ]:
# This calls the function from the beginning which joins a shapefile dataset to a linestring set
joined_svi_df = find_nearest_linestring_efficient(joined_svi_address_df, roads_df_transformed, 200)


Matched 0 out of 0 points


<ipython-input-3-0ee733655df3>:24: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  print(f"Matched {result.notna().any(axis=1).sum()} out of {len(result)} points")


In [ ]:
# This calls the function from the beginning which joins a shapefile dataset to a linestring set
joined_amenities_df = find_nearest_linestring_efficient(amenities, roads_df_transformed, 200)


Matched 0 out of 0 points


<ipython-input-3-0ee733655df3>:24: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  print(f"Matched {result.notna().any(axis=1).sum()} out of {len(result)} points")


In [ ]:
# Filter to my address (for testing)

selected_records = final_results[(final_results['street'] == 'TUCKER ST')] # & (final_results['number'] == '335')]


In [ ]:
# Export the selected_record dataframe above to a csv file

selected_records.to_csv('selected_records.csv', index=False)


<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
